# FBMN Annotation & Visualization in Cytoscape

<details>
    <summary>Click to view the narrative</summary>

## Overall narrative of the workshop

- import a network (GNPS or MSQuery) in Cytoscape

- import the MZmine table for node metadata

- import the IIMN edge table.

- show the styling with ion identity network.

- duplicate the views and rename the views for each tool.

- explain requirements for further import (unique feature id, first column, smiles [name and ,] ). And opportunity for information-rich labeling for each tool. 

- import tool prepared tool output into the network.

- bring-in styles (Get the Cytoscape style for each tool).

- explore with each tool
</details>

## Let's go

**Installation**: install Cytoscape [https://cytoscape.org/](https://cytoscape.org/) and its ChemViz2 pluggin [https://apps.cytoscape.org/apps/chemviz2](https://apps.cytoscape.org/apps/chemviz2).

**Notebook**: the notebook is available at [https://github.com/lfnothias/FBMN_annotation_fusion_visualization_Cytoscape](https://github.com/lfnothias/FBMN_annotation_fusion_visualization_Cytoscape). It can run as a Binder instance (see link).

**Input files**: the FBMN network and annotation files were uploaded to the github repo as of April 15th 2024. If needed, update them.

## Step 1 - Import a network

- Open Cytoscape and import a FBMN derived network file (graphml file). The process can be done by drag & drop or with `File / Import / Network from File`.

- Demo about Cytoscape and key definitions (node table, edge table).

- We observe the limited number of information available in the file.

## Step 2 - Import a Node Table

Node table contains metadata about nodes (a feature MS1/MS2 spectra) and are mapped with column containing the 'Feature_ID' (or its variant = ['scan',featureID', 'feature_id', ... ) on the 'shared name' column. Preferably on the first column of the node table (or select the mapping key accordingly). It must be unique. If columns are already present, they will be ovewritten.

- We will import the `mzmine_results_iimn_gnps_quant.csv` into Cytoscape. First lets take a look at its content and comment.

In [1]:
import pandas as pd 
mzmine = 'results_download/mzmine/mzmine_results_iimn_gnps_quant.csv'
pd.read_csv(mzmine).head(5)

,row ID,row m/z,row retention time,row ion mobility,row ion mobility unit,row CCS,correlation group ID,annotation network number,best ion,auto MS2 verify,...,171103_PMA_TK_PA14_06.mzML Peak area,171103_PMA_TK_M1_04.mzML Peak area,171103_PMA_TK_QC_06.mzML Peak area,171103_PMA_TK_QC_08.mzML Peak area,171103_PMA_TK_PA14_02.mzML Peak area,171103_PMA_TK_M1_03.mzML Peak area,171103_PMA_TK_M1_05.mzML Peak area,171103_PMA_TK_M1_06.mzML Peak area,171103_PMA_TK_M1_01.mzML Peak area,Unnamed: 38
0,2627,120.080494,1.093362,NaN,NaN,NaN,295.0,NaN,NaN,NaN,...,56583.450,3833.324,29376.084,28985.492,75136.660,4629.6140,4354.288,6301.5215,4848.9614,NaN
1,3923,120.080579,1.516975,NaN,NaN,NaN,15.0,NaN,NaN,NaN,...,16967.299,5470.070,20291.752,14098.369,20508.709,8007.4146,6572.576,10250.7780,8709.6860,NaN
2,2034,120.080812,0.933045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17117.354,0.000,11344.498,9181.703,11280.098,0.0000,0.000,0.0000,0.0000,NaN
3,1325,120.080901,0.702407,NaN,NaN,NaN,163.0,NaN,NaN,NaN,...,2677746.800,97282.740,1343596.600,1310642.200,3106760.500,98606.8360,98289.110,142277.6700,141920.5500,NaN
4,7607,122.547348,8.412525,NaN,NaN,NaN,21.0,NaN,NaN,NaN,...,869530.560,877802.000,807413.440,916807.750,809052.200,815866.6000,759270.250,885191.9400,913430.2000,NaN


### Lets import it in Cytoscape 

- The node table import can be done with  `File / Import / Table from File`. \
Make sure you select the correct the right Network for import (Network Collection) and the correct Data Type (Node or Edge Table).

- Start basic Node styling with the Style panel.

## Step 3 - Import an Edge Table

Edge table contains metadata about nodes (a feature MS1/MS2 spectra) and are mapped with column containing the interaction ID [NodeID1 (-) NodeID2) and named 'shared name'. Preferably on the first column of the node table (or select the mapping key accordingly). It must be unique. If columns are already present, they will be ovewritten.


### Prepare IIMN edges for Cytoscape

We need to create a 'shared name' column in the edge table to streamline Cytoscape import.

Original table:
| ID1 | ID2 | EdgeType | Score | Annotation |
|-----|-----|----------|-------|------------|
| 36  | 37  | MS1 annotation | 2      | [M+K]+ [2M+Na]+ dm/z=299.18058           |

Prepared table:
| shared name | ID1 | ID2 | EdgeType | Score | Annotation |
|-------------|-----|-----|----------|-------|------------|
| 36 (-) 37   | 36  | 37  |MS1 annotation| 2      | [M+K]+ [2M+Na]+ dm/z=299.18058           |


In [2]:
def detect_separator(filename):
    with open(filename, 'r') as file:
        first_line = file.readline()
        if '\t' in first_line:
            return '\t'
        else:
            return ','  # Default to comma if no tab found

def prepare_fbmn_iimn_edge_annotation_cytoscape(input_file, output_suffix='_prep'):
    # Load the CSV file into a DataFrame
    separator = detect_separator(input_file)
    df = pd.read_csv(input_file, sep=separator)

    # Check if ID1 and ID2 are in the DataFrame
    if 'ID1' not in df.columns or 'ID2' not in df.columns:
        raise ValueError("ID1 or ID2 column is missing in the DataFrame.")

    # Create a new column 'ID1-ID2' by concatenating 'ID1' and 'ID2' with " (-) " in between
    df['shared name'] = df['ID1'].astype(str) + ' (-) ' + df['ID2'].astype(str)

    # Move the new column to the first position
    cols = df.columns.tolist()
    cols = ['shared name'] + [col for col in cols if col != 'shared name']
    df = df[cols]

    # Save the modified DataFrame to a new CSV file with the specified suffix
    output_file = f"{input_file.rsplit('.', 1)[0]}{output_suffix}.tsv"
    df.to_csv(output_file, index=False, sep='\t')

    # Print the new output file name
    print(f"File saved as: {output_file}")
    return df.head(5)

In [3]:
iimn_edge_table_path = 'results_download/mzmine/mzmine_results_iimn_gnps_edges_msannotation.csv'
prepare_fbmn_iimn_edge_annotation_cytoscape(iimn_edge_table_path)

File saved as: results_download/mzmine/mzmine_results_iimn_gnps_edges_msannotation_prep.tsv


,shared name,ID1,ID2,EdgeType,Score,Annotation
0,36 (-) 37,36,37,MS1 annotation,2,[M+K]+ [2M+Na]+ dm/z=299.18058
1,45 (-) 46,45,46,MS1 annotation,2,[M+Na]+ [M+H]+ dm/z=21.98143
2,67 (-) 104,67,104,MS1 annotation,5,[M-H+2Na]+ [2M+H]+ dm/z=85.07864
3,67 (-) 98,67,98,MS1 annotation,5,[M+Na]+ [2M+H]+ dm/z=107.0606
4,97 (-) 239,97,239,MS1 annotation,5,[M+H]+ [2M-H+2Na]+ dm/z=173.00623


### Import the prepared IIMN Edge Table into Cytoscape

- The node table import can be done with  `File / Import / Table from File`.\
Make sure you select the correct the right Network for import (Network Collection) and the correct Data Type (Node or Edge Table).

- Start styling Edges with the Style panel. 

## Step 4 - Importing other annotations

### MS2Query annotations

Lets take a look at MS2Query table and import it 

In [4]:
ms2query_annotation_path = 'results_download/matchms/results_for_cytoscape/ms2query_results_for_cytoscape.csv'
pd.read_csv(ms2query_annotation_path).head(5)

,feature_id,ms2query_model_prediction,precursor_mz_difference,precursor_mz_query_spectrum,precursor_mz_analog,inchikey,analog_compound_name,smiles,retention_time,cf_kingdom,cf_superclass,cf_class,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,ms2query_reliable_annotation
0,2627,0.8050,0.0005,120.0805,120.081,LPAGFVYQRIESJQ,Indolin,C1CC2=CC=CC=C2N1,65.602,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Indolines,Indolines,Simple indole alkaloids,Tryptophan alkaloids,Alkaloids,YES
1,3923,0.8304,0.0004,120.0806,120.081,LPAGFVYQRIESJQ,"2,3-dihydro-1H-indole",C1CC2=CC=CC=C2N1,91.019,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Indolines,Indolines,Simple indole alkaloids,Tryptophan alkaloids,Alkaloids,YES
2,2034,0.8412,0.0002,120.0808,120.081,LPAGFVYQRIESJQ,Indolin,C1CC2=CC=CC=C2N1,55.983,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Indolines,Indolines,Simple indole alkaloids,Tryptophan alkaloids,Alkaloids,YES
3,1325,0.8408,0.0001,120.0809,120.081,LPAGFVYQRIESJQ,"2,3-dihydro-1H-indole",C1CC2=CC=CC=C2N1,42.144,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Indolines,Indolines,Simple indole alkaloids,Tryptophan alkaloids,Alkaloids,YES
4,57,0.8032,0.0005,124.0395,124.040,TWBYWOBDOCUKOW,4-Carboxypyridine,OC(=O)c(c1)ccnc1,19.838,Organic compounds,Organoheterocyclic compounds,Pyridines and derivatives,Pyridinecarboxylic acids and derivatives,Pyridinecarboxylic acids,Pyridine alkaloids,Nicotinic acid alkaloids,Alkaloids,YES


It has the requirements for Cytoscape import on FBMN network !

**Requirements**:
- a `feature_id` column from the FBMN. The exact column naming is flexible. 
- the `feature_id` entries must be unique and consistent.

    
**Bonus to streamline Cytoscape**:
- The `feature_id` column (or equivalent) should be the first column.
- If there are structural annotations, the annotation name should be in a `name` column, the smiles should be in a `smiles` column.
- We introduce a prefix for all the annotation columns.
- We will add a 'annotation' tool column for visualization.


Lets check the minimum requirements

In [5]:
ms2query_annotation = pd.read_csv(ms2query_annotation_path)
print(ms2query_annotation.columns)
ms2query_annotation.head(2)

Index(['feature_id', 'ms2query_model_prediction', 'precursor_mz_difference',
       'precursor_mz_query_spectrum', 'precursor_mz_analog', 'inchikey',
       'analog_compound_name', 'smiles', 'retention_time', 'cf_kingdom',
       'cf_superclass', 'cf_class', 'cf_subclass', 'cf_direct_parent',
       'npc_class_results', 'npc_superclass_results', 'npc_pathway_results',
       'ms2query_reliable_annotation'],
      dtype='object')


,feature_id,ms2query_model_prediction,precursor_mz_difference,precursor_mz_query_spectrum,precursor_mz_analog,inchikey,analog_compound_name,smiles,retention_time,cf_kingdom,cf_superclass,cf_class,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,ms2query_reliable_annotation
0,2627,0.8050,0.0005,120.0805,120.081,LPAGFVYQRIESJQ,Indolin,C1CC2=CC=CC=C2N1,65.602,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Indolines,Indolines,Simple indole alkaloids,Tryptophan alkaloids,Alkaloids,YES
1,3923,0.8304,0.0004,120.0806,120.081,LPAGFVYQRIESJQ,"2,3-dihydro-1H-indole",C1CC2=CC=CC=C2N1,91.019,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Indolines,Indolines,Simple indole alkaloids,Tryptophan alkaloids,Alkaloids,YES


#### We check for uniqueness of feature_id

In [6]:
# Check for if feature_id is unique.
print('Is feature_id unique ? = ' + str(ms2query_annotation['feature_id'].is_unique))

# Lets rename the name and same
ms2query_annotation['name'] = ms2query_annotation['analog_compound_name']
ms2query_annotation.to_csv(ms2query_annotation_path[:-4]+'_prep.tsv', sep='\t', index=False)

Is feature_id unique ? = True


### Lets import it in Cytoscape

- The node table import can be done with  `File / Import / Table from File`. \
Make sure you select the correct the right Network for import (Network Collection) and the correct Data Type (Node or Edge Table).

Lets Customize the node style. Lets search with the filter function.

### Lets do the same with another table

#### MassQL annotation



In [7]:
# Load the CSV file into a DataFrame
tool = 'MassQL'
massql_annotations = 'results_download/GNPS2/ea4293bedd5440148267cb201ef7edbc-merged_query_results_MassQL.tsv'
df = pd.read_csv(massql_annotations, sep='\t')
print(df.columns)
df.head(3)

Index(['charge', 'filename', 'i', 'i_norm', 'i_norm_ms1', 'ms1scan', 'mslevel',
       'original_path', 'precmz', 'query_index', 'rt', 'scan'],
      dtype='object')


,charge,filename,i,i_norm,i_norm_ms1,ms1scan,mslevel,original_path,precmz,query_index,rt,scan
0,1,mzmine_results_iimn_gnps.mgf,125140000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,274.1435,0,0,5051
1,1,mzmine_results_iimn_gnps.mgf,74160000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,274.1434,0,0,5081
2,1,mzmine_results_iimn_gnps.mgf,106940000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,276.1591,0,0,5179


#### MassQL table

Lets open the MassQL table 

Original table
| charge | filename            | i     | i_norm | scan | ... |
|---------------|----------------------------|--------------|---------------|---------------|---------------|
| 1        | mzmine_results_iimn_gnps.mgf                       |  125140000.0 | 1.0 | 5051| ... |

Prepared table
| scan | MassQL_charge | MassQL_filename     | MassQL_i     | MassQL_i_norm | ... |
|---------------|----------------------------|--------------|---------------|---------------|--------------|
| 5051 | 1         | mzmine_results_iimn_gnps.mgf                         | 125140000.0 | 1.0 | ... |

### Helpers function for preparing the annotation table

In [8]:
def detect_separator(filename):
    with open(filename, 'r') as file:
        first_line = file.readline()
        if '\t' in first_line:
            return '\t'
        else:
            return ','  # Default to comma if no tab found


def duplicate_column_if_string_found(df, substring, new_column_name):
    # Track if a column was found and duplicated
    found_and_duplicated = False

    # Loop through all column names in the DataFrame
    for col in df.columns:
        # Check if the substring matches part of any column name
        if substring.lower() in col.lower():  # This makes the search case-insensitive
            # Create a new column name by appending the specified new column name
            # Duplicate the column
            df[new_column_name] = df[col].astype(str).replace('Spectral Match to ', '', regex=True)
            print(f"Column '{col}' duplicated into '{new_column_name}'.")
            found_and_duplicated = True

    # If no column matches the substring, print a message
    if not found_and_duplicated:
        print(f"No columns found containing the substring '{substring}'.")


def aggregate_columns(series):
    # Convert all entries to strings, ensure they are unique and handle NaN values
    sorted_values = sorted(series.dropna().astype(str))
    aggregated_string = ','.join(sorted(sorted_values))
    return aggregated_string


def prepare_fbmn_annotation_for_cytoscape(input_file, feature_id_column, tool_prefix, output_suffix='_prep'):
    # Load the CSV file into a DataFrame
    separator = detect_separator(input_file)
    df = pd.read_csv(input_file, sep=separator)

    # Drop columns where name contains 'Unnamed'
    df = df.loc[:, ~df.columns.str.contains('Unnamed')]

    # Identify any column containing 'smiles' in its name, case-insensitively
    smiles_columns = [col for col in df.columns if 'smiles' in col.lower()]

    # Case insensitive check for 'compound_name' or 'name'
    lower_columns = {col.lower(): col for col in df.columns}  # Create a dict with lower case keys and original column names as values
    compound_col = lower_columns.get('compound_name', lower_columns.get('name'))


    # Check for and remove duplicates based on feature_id_column with either compound_name or smiles
    if compound_col:
        # Create a combined duplicate check list
        for smiles_col in smiles_columns:
            # Remove duplicates where the feature ID and either the compound name or one of the smiles columns are the same
            initial_row_count = len(df)
            df.drop_duplicates(subset=[feature_id_column, compound_col], keep='first', inplace=True)
            df.drop_duplicates(subset=[feature_id_column, smiles_col], keep='first', inplace=True)
            final_row_count = len(df)
            print(f"Removed {initial_row_count - final_row_count} duplicates based on {feature_id_column}, {compound_col}, and {smiles_col}")

    else:
        print("Neither 'compound_name' nor 'name' column is present. Checking for duplicates based on SMILES only.")
        for smiles_col in smiles_columns:
            initial_row_count = len(df)
            df.drop_duplicates(subset=[feature_id_column, smiles_col], keep='first', inplace=True)
            final_row_count = len(df)
            print(f"Removed {initial_row_count - final_row_count} duplicates based on {feature_id_column} and {smiles_col}")

    # Check existence of expected columns
    expected_cols = ['score', 'adduct', 'mol_formula', 'inchi', 'inchi_key', 'compound_name']
    smiles_like_cols = [col for col in df.columns if 'smiles' in col.lower()]
    cols_to_aggregate = expected_cols + smiles_like_cols
    cols_to_aggregate = [col for col in cols_to_aggregate if col in df.columns]

    # If feature_id_column is present and not unique, handle aggregation
    if feature_id_column in df.columns:
            if not df[feature_id_column].is_unique:
                grouped = df.groupby(feature_id_column)[cols_to_aggregate].agg(aggregate_columns).reset_index()
                # Drop the original aggregated columns from main DataFrame and merge with aggregated data
                df = df.drop(columns=cols_to_aggregate).drop_duplicates(subset=feature_id_column)
                df = pd.merge(df, grouped, on=feature_id_column, how='left')
                print('Aggregation completed. FeatureID had duplicates.')
            else:
                print('FeatureID is unique. No aggregation needed.')
    else:
        raise ValueError(f"{feature_id_column} is not a column in the DataFrame.")

    # Copy 'smiles' columns with prefixes and keep original
    for smiles_column in smiles_columns:
        prefixed_smiles_column = f"{tool_prefix}_{smiles_column}"
        if prefixed_smiles_column not in df.columns:  # Check if prefixed column already exists
            df[prefixed_smiles_column] = df[smiles_column]

    # Prepare to add prefix to all columns except feature_id_column and original smiles_columns
    rename_dict = {}
    for col in df.columns:
        if col not in smiles_columns and col != feature_id_column and not col.startswith(tool_prefix):
            rename_dict[col] = f"{tool_prefix}_{col}"

    df.rename(columns=rename_dict, inplace=True)

    # Add an extra column 'annotation_tool' with the value of the tool prefix
    df['annotation_tool'] = tool_prefix

    duplicate_column_if_string_found(df, 'Compound_name', 'name')

    # Renaming and moving feature_id_column to the first position
    df = df[[feature_id_column] + [col for col in df.columns if col != feature_id_column]]  # This moves the feature_id_column to the first position

    # Special handling if tool are for sirius
    if tool_prefix.lower() == 'sir_class':
        # Check if necessary columns exist
        if 'sir_class_NPC#class' in df.columns and 'sir_class_molecularFormula' in df.columns and 'sir_class_adduct' in df.columns:
            df['name'] = df['sir_class_NPC#class'] + ' | ' + df['sir_class_molecularFormula'] + ' | ' + df['sir_class_adduct']
        else:
            print("Required Canopus columns are not all present.")

    elif tool_prefix.lower() == 'sir_struct':
        # Check if necessary columns exist
        if 'sir_struct_name' in df.columns and 'sir_struct_ConfidenceScore' in df.columns:
            df['name'] = df['sir_struct_name'] + ' (' + df['sir_struct_ConfidenceScore'].astype(str) + ')'
        else:
            print("Required Sirius columns are not all present.")

    elif tool_prefix.lower() == 'tima':
        # Check if necessary columns exist
        if 'tima_candidate_structure_name' in df.columns and 'tima_candidate_structure_tax_npc_03cla' in df.columns:
            df['name'] = df['tima_candidate_structure_name'] + ' | ' +df['tima_candidate_structure_tax_npc_03cla']

        else:
            print("Required TIMA columns are not all present.")

        smiles_col = 'tima_candidate_structure_smiles_no_stereo'
        if smiles_col in df.columns:
            df['smiles'] = df[smiles_col].str.replace('|', ',', regex=False)

    # Save the modified DataFrame to a new CSV file with the specified suffix
    output_file = f"{input_file.rsplit('.', 1)[0]}{output_suffix}.tsv"
    df.to_csv(output_file, index=False, sep='\t')

    # Print the new output file name
    print(f"File saved as: {output_file}")
    return df.head(5)


Lets process MassQL table

In [9]:
prepare_fbmn_annotation_for_cytoscape(massql_annotations, 'scan', tool)

Neither 'compound_name' nor 'name' column is present. Checking for duplicates based on SMILES only.
FeatureID is unique. No aggregation needed.
No columns found containing the substring 'Compound_name'.
File saved as: results_download/GNPS2/ea4293bedd5440148267cb201ef7edbc-merged_query_results_MassQL_prep.tsv


,scan,MassQL_charge,MassQL_filename,MassQL_i,MassQL_i_norm,MassQL_i_norm_ms1,MassQL_ms1scan,MassQL_mslevel,MassQL_original_path,MassQL_precmz,MassQL_query_index,MassQL_rt,annotation_tool
0,5051,1,mzmine_results_iimn_gnps.mgf,125140000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,274.1435,0,0,MassQL
1,5081,1,mzmine_results_iimn_gnps.mgf,74160000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,274.1434,0,0,MassQL
2,5179,1,mzmine_results_iimn_gnps.mgf,106940000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,276.1591,0,0,MassQL
3,5233,1,mzmine_results_iimn_gnps.mgf,98180000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,258.1489,0,0,MassQL
4,5425,1,mzmine_results_iimn_gnps.mgf,98290000.0,1.0,NaN,0,2,input_spectra/mzmine_results_iimn_gnps.mgf,274.1436,0,0,MassQL


#### Lets import MassQL Node Table in Cytoscape

### Lets prepare the GNPS table

In [10]:
# Load the CSV file into a DataFrame
tool = 'GNPS'
gnps_annotations = 'results_download/GNPS2/861f707d5a4f42e88486c77a4693a38d-merged_results_with_gnps.tsv'
df = pd.read_csv(gnps_annotations, sep='\t')
df.columns

#We will add a 'tool' prefix to the column and move the 'scan' column to the first position.

Index(['SpectrumID', '#Scan#', 'SpectrumFile', 'LibraryName', 'MQScore',
       'TIC_Query', 'RT_Query', 'MZErrorPPM', 'SharedPeaks', 'MassDiff',
       'SpecMZ', 'SpecCharge', 'FileScanUniqueID', 'NumberHits',
       'Compound_Name', 'Ion_Source', 'Instrument', 'Compound_Source', 'PI',
       'Data_Collector', 'Adduct', 'Precursor_MZ', 'ExactMass', 'Charge',
       'CAS_Number', 'Pubmed_ID', 'Smiles', 'INCHI', 'INCHI_AUX',
       'Library_Class', 'IonMode', 'Organism', 'LibMZ', 'UpdateWorkflowName',
       'LibraryQualityString', 'tags', 'molecular_formula', 'InChIKey',
       'InChIKey-Planar', 'superclass', 'class', 'subclass',
       'npclassifier_superclass', 'npclassifier_class',
       'npclassifier_pathway'],
      dtype='object')

In [11]:
prepare_fbmn_annotation_for_cytoscape(gnps_annotations, '#Scan#', tool)

Removed 0 duplicates based on #Scan#, Compound_Name, and Smiles
FeatureID is unique. No aggregation needed.
Column 'GNPS_Compound_Name' duplicated into 'name'.
File saved as: results_download/GNPS2/861f707d5a4f42e88486c77a4693a38d-merged_results_with_gnps_prep.tsv


,#Scan#,GNPS_SpectrumID,GNPS_SpectrumFile,GNPS_LibraryName,GNPS_MQScore,GNPS_TIC_Query,GNPS_RT_Query,GNPS_MZErrorPPM,GNPS_SharedPeaks,GNPS_MassDiff,...,GNPS_InChIKey-Planar,GNPS_superclass,GNPS_class,GNPS_subclass,GNPS_npclassifier_superclass,GNPS_npclassifier_class,GNPS_npclassifier_pathway,GNPS_Smiles,annotation_tool,name
0,322,CCMSLIB00003137613,spectra_filtered.mgf,GNPS-NIST14-MATCHES.mgf,0.998956,199600000.0,0,0.586615,7,0.000107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GNPS,L-Tyrosine from NIST14
1,61,CCMSLIB00003138270,spectra_filtered.mgf,GNPS-NIST14-MATCHES.mgf,0.998848,79300000.0,0,0.609937,8,0.000107,...,ODKSFYDXXFIFQN,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Small peptides,Aminoacids,Amino acids and Peptides,C(C[C@@H](C(=O)O)N)CN=C(N)N,GNPS,L-Arginine from NIST14
2,651,CCMSLIB00003137613,spectra_filtered.mgf,GNPS-NIST14-MATCHES.mgf,0.998737,109200000.0,0,1.173230,7,0.000214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GNPS,L-Tyrosine from NIST14
3,3836,CCMSLIB00003136674,spectra_filtered.mgf,GNPS-NIST14-MATCHES.mgf,0.998349,167100000.0,0,1.248390,9,0.000427,...,JNTMAZFVYNDPLB,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Small peptides,Dipeptides,Amino acids and Peptides,CC[C@H](C)[C@@H](C(=O)N1CCC[C@H]1C(=O)N[C@@H](...,GNPS,Ile-Pro-Ile from NIST14
4,1406,CCMSLIB00010013036,spectra_filtered.mgf,GNPS-LIBRARY.mgf,0.998292,142500000.0,0,1.043430,10,0.000198,...,FWULQXYJOANGSS,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Small peptides,Dipeptides,Amino acids and Peptides,O=C(O)C(NC(CCC)=O)CCO,GNPS,butyrylhomoserine


### Lets prepare the MZmine spectral library annotation table

- an `id` column but are not unique -> we can concatenate them

In [12]:
# Load the CSV file into a DataFrame
tool = 'MZmine'
mzmine_annotations = 'results_download/mzmine/mzmine_results_annotations.csv'
df = pd.read_csv(mzmine_annotations, sep=',')
df.columns
df.head(2)

,id,compound_name,adduct,score,precursor_mz,ion_mobility,ccs,rt,mol_formula,smiles,inchi,inchi_key,method
0,57,"Nicotinic acid, pyridine-3-carboxylic acid",[M+H]+,0.975,124.0393,NaN,NaN,NaN,C6H5NO2,OC(=O)C1=CC=CN=C1,"InChI=1S/C6H5NO2/c8-6(9)5-2-1-3-7-4-5/h1-4H,(H...",PVNIIMVLHYAWGP-UHFFFAOYSA-N,spectral_library_annotation
1,57,"Nicotinic acid, pyridine-3-carboxylic acid",[M+H]+,0.975,124.0393,NaN,NaN,NaN,C6H5NO2,OC(=O)C1=CC=CN=C1,"InChI=1S/C6H5NO2/c8-6(9)5-2-1-3-7-4-5/h1-4H,(H...",PVNIIMVLHYAWGP-UHFFFAOYSA-N,spectral_library_annotation


Original table
| id | compound_name            | adduct    | score | scan | ... |smiles |... |
|---------------|----------------------------|--------------|---------------|---------------|---------------|---------------|---------------|
| 57        | Nicotinic acid, pyridine-3-carboxylic acid                      | [M+H]+ | 1.0 | 0.975| ... |	OC(=O)C1=CC=CN=C1 | ... | 
| 57        | Nicotinic acid, pyridine-3-carboxylic acid                      |  [M+H]+ | 1.0 | 0.975| ... | OC(=O)C1=CC=CN=C1 |... |
| 57        | Isonicotinic acid                     |  [M+H]+ | 1.0 | 0.874| ... | c1cnccc1C(=O)O | ... |

Prepared table
| id | compound_name            | adduct    | score | scan | ... |smiles |... |
|---------------|----------------------------|--------------|---------------|---------------|---------------|---------------|---------------|
| 57        | Nicotinic acid, pyridine-3-carboxylic acid,  Isonicotinic acid | [M+H]+,[M+H]+ | 1.0, 1.0 | 0.975, 0.874| ... |	OC(=O)C1=CC=CN=C1,c1cnccc1C(=O)O | ... | 

In [13]:
prepare_fbmn_annotation_for_cytoscape(mzmine_annotations, 'id', tool)

Removed 388 duplicates based on id, compound_name, and smiles
Aggregation completed. FeatureID had duplicates.
Column 'MZmine_compound_name' duplicated into 'name'.
File saved as: results_download/mzmine/mzmine_results_annotations_prep.tsv


,id,MZmine_precursor_mz,MZmine_ion_mobility,MZmine_ccs,MZmine_rt,MZmine_method,MZmine_score,MZmine_adduct,MZmine_mol_formula,MZmine_inchi,MZmine_inchi_key,MZmine_compound_name,smiles,MZmine_smiles,annotation_tool,name
0,57,124.039300,NaN,NaN,NaN,spectral_library_annotation,0.975,[M+H]+,C6H5NO2,"InChI=1S/C6H5NO2/c8-6(9)5-2-1-3-7-4-5/h1-4H,(H...",PVNIIMVLHYAWGP-UHFFFAOYSA-N,"Nicotinic acid, pyridine-3-carboxylic acid",OC(=O)C1=CC=CN=C1,OC(=O)C1=CC=CN=C1,MZmine,"Nicotinic acid, pyridine-3-carboxylic acid"
1,61,175.118958,NaN,NaN,12.6500,spectral_library_annotation,"0.983,0.992","[M+H]+,[M+H]+","C6H14N4O2,C6H14N4O2",InChI=1S/C6H14N4O2/c7-4(5(11)12)2-1-3-10-6(8)9...,"ODKSFYDXXFIFQN-BYPYZUCNSA-N,ODKSFYDXXFIFQN-UHF...","Arginine, 2-amino-5-(diaminomethylideneamino)p...","C(CC(C(=O)O)N)CN=C(N)N,O=C(O)C(N)CCCNC(=N)N","C(CC(C(=O)O)N)CN=C(N)N,O=C(O)C(N)CCCNC(=N)N",MZmine,"Arginine, 2-amino-5-(diaminomethylideneamino)p..."
2,132,150.058330,NaN,NaN,NaN,spectral_library_annotation,"0.977,0.984,0.991","[M+H]+,[M+H]+,[M+H]+","C5H11NO2S,C5H11NO2S,C5H11NO2S","InChI=1S/C5H11NO2S/c1-9-3-2-4(6)5(7)8/h4H,2-3,...","FFEARJCKVFRZRR-BYPYZUCNSA-N,FFEARJCKVFRZRR-BYP...","Methionine,L-METHIONINE,L-Methionine","CSCCC(C(=O)O)N,CSCC[C@@H](C(=O)O)N,O=C(O)C(N)CCSC","CSCCC(C(=O)O)N,CSCC[C@@H](C(=O)O)N,O=C(O)C(N)CCSC",MZmine,"Methionine,L-METHIONINE,L-Methionine"
3,239,130.050000,NaN,NaN,8.7083,spectral_library_annotation,0.968,[M+H]+,C5H7NO3,"InChI=1S/C5H7NO3/c7-4-2-1-3(6-4)5(8)9/h3H,1-2H...",ODHCTXKNWHHXJC-VKHMYHEASA-N,Pyroglutamic acid,O=C(O)C1N=C(O)CC1,O=C(O)C1N=C(O)CC1,MZmine,Pyroglutamic acid
4,298,190.070999,NaN,NaN,NaN,spectral_library_annotation,0.948,[M+H]+,C7H11NO5,InChI=1S/C7H11NO5/c1-4(9)8-5(7(12)13)2-3-6(10)...,RFMMMVDNIPUKGG-UHFFFAOYSA-N,N-ACETYL-GLUTAMIC ACID,O=C(O)CCC(N=C(O)C)C(=O)O,O=C(O)CCC(N=C(O)C)C(=O)O,MZmine,N-ACETYL-GLUTAMIC ACID


## SIRIUS annotations

Lets see SIRIUS class and structure annotation.

#### SIRIUS class annnotation

There is a 'featureId' column. 

Lets do some bonus formatting

In [14]:
tool = 'sir_class'
class_annotations_path = 'results_download/sirius/summary-files/canopus_structure_summary.tsv'
pd.read_csv(class_annotations_path, sep= '\t').head(5)

,formulaRank,molecularFormula,adduct,precursorFormula,NPC#pathway,NPC#pathway Probability,NPC#superclass,NPC#superclass Probability,NPC#class,NPC#class Probability,...,ClassyFire#class,ClassyFire#class Probability,ClassyFire#superclass,ClassyFire#superclass probability,ClassyFire#all classifications,ionMass,retentionTimeInSeconds,formulaId,alignedFeatureId,featureId
0,1,C9H18N2O4,[M + H]+,C9H19N2O4+,Amino acids and Peptides,1.000,Small peptides,1.000,Dipeptides,0.991,...,Carboxylic acids and derivatives,1.0,Organic acids and derivatives,1.0,Organic compounds; Lipids and lipid-like molec...,219.134,20,567749660631298519,567749454112148954,14
1,1,C11H23N5O3,[M + H]+,C11H24N5O3+,Amino acids and Peptides,1.000,Small peptides,1.000,Dipeptides,1.000,...,Carboxylic acids and derivatives,1.0,Organic acids and derivatives,1.0,Organic compounds; Lipids and lipid-like molec...,274.187,22,567749695871841810,567749454275726814,28
2,1,C12H21N5O5,[M + Na]+,C12H21N5NaO5+,Amino acids and Peptides,0.999,Small peptides,1.000,Dipeptides,0.991,...,Carboxylic acids and derivatives,1.0,Organic acids and derivatives,1.0,Organic compounds; Lipids and lipid-like molec...,338.143,19,567749669149930142,567749454372195810,33
3,1,C10H20N2O4S,[M + H]+,C10H21N2O4S+,Amino acids and Peptides,1.000,Small peptides,1.000,Dipeptides,0.991,...,Carboxylic acids and derivatives,1.0,Organic acids and derivatives,1.0,Organic compounds; Organosulfur compounds; Lip...,265.122,20,567749657737228665,567749454531579370,45
4,1,C16H30N6O4,[M + H]+,C16H31N6O4+,Amino acids and Peptides,1.000,Small peptides,0.999,Tripeptides,0.931,...,Carboxylic acids and derivatives,1.0,Organic acids and derivatives,1.0,Organic compounds; Organoheterocyclic compound...,371.240,20,567749662992691702,567749454649019886,55


In [15]:
prepare_fbmn_annotation_for_cytoscape(class_annotations_path, 'featureId', tool)

Neither 'compound_name' nor 'name' column is present. Checking for duplicates based on SMILES only.
Aggregation completed. FeatureID had duplicates.
No columns found containing the substring 'Compound_name'.
File saved as: results_download/sirius/summary-files/canopus_structure_summary_prep.tsv


,featureId,sir_class_formulaRank,sir_class_molecularFormula,sir_class_precursorFormula,sir_class_NPC#pathway,sir_class_NPC#pathway Probability,sir_class_NPC#superclass,sir_class_NPC#superclass Probability,sir_class_NPC#class,sir_class_NPC#class Probability,...,sir_class_ClassyFire#superclass,sir_class_ClassyFire#superclass probability,sir_class_ClassyFire#all classifications,sir_class_ionMass,sir_class_retentionTimeInSeconds,sir_class_formulaId,sir_class_alignedFeatureId,sir_class_adduct,annotation_tool,name
0,14,1,C9H18N2O4,C9H19N2O4+,Amino acids and Peptides,1.000,Small peptides,1.000,Dipeptides,0.991,...,Organic acids and derivatives,1.0,Organic compounds; Lipids and lipid-like molec...,219.134,20,567749660631298519,567749454112148954,[M + H]+,sir_class,Dipeptides | C9H18N2O4 | [M + H]+
1,28,1,C11H23N5O3,C11H24N5O3+,Amino acids and Peptides,1.000,Small peptides,1.000,Dipeptides,1.000,...,Organic acids and derivatives,1.0,Organic compounds; Lipids and lipid-like molec...,274.187,22,567749695871841810,567749454275726814,[M + H]+,sir_class,Dipeptides | C11H23N5O3 | [M + H]+
2,33,1,C12H21N5O5,C12H21N5NaO5+,Amino acids and Peptides,0.999,Small peptides,1.000,Dipeptides,0.991,...,Organic acids and derivatives,1.0,Organic compounds; Lipids and lipid-like molec...,338.143,19,567749669149930142,567749454372195810,[M + Na]+,sir_class,Dipeptides | C12H21N5O5 | [M + Na]+
3,45,1,C10H20N2O4S,C10H21N2O4S+,Amino acids and Peptides,1.000,Small peptides,1.000,Dipeptides,0.991,...,Organic acids and derivatives,1.0,Organic compounds; Organosulfur compounds; Lip...,265.122,20,567749657737228665,567749454531579370,[M + H]+,sir_class,Dipeptides | C10H20N2O4S | [M + H]+
4,55,1,C16H30N6O4,C16H31N6O4+,Amino acids and Peptides,1.000,Small peptides,0.999,Tripeptides,0.931,...,Organic acids and derivatives,1.0,Organic compounds; Organoheterocyclic compound...,371.240,20,567749662992691702,567749454649019886,[M + H]+,sir_class,Tripeptides | C16H30N6O4 | [M + H]+


#### SIRIUS structure annotations

There is a `feature_id` column. 

Lets do some bonus formatting

In [16]:
tool = 'sir_struct'
sirius_annotations_path = 'results_download/sirius/summary-files/structure_identifications.tsv'
pd.read_csv(sirius_annotations_path, sep= '\t').head(5)

,structurePerIdRank,formulaRank,ConfidenceScoreExact,ConfidenceScoreApproximate,CSI:FingerIDScore,ZodiacScore,SiriusScore,molecularFormula,adduct,precursorFormula,...,smiles,xlogp,pubchemids,links,dbflags,ionMass,retentionTimeInSeconds,formulaId,alignedFeatureId,featureId
0,1,1,0.739409,0.739409,-15.718746,nul,38.171,C9H18N2O4,[M + H]+,C9H19N2O4+,...,CC(C)C(C(=O)NC(C(C)O)C(=O)O)N,-0.997,4046492;9815826;124306824;124422925;156908028;...,COCONUT:(CNP0292075);PUBCHEMANNOTATIONBIO;HMDB...,556793898,219.134,20,567749660631298519,567749454112148954,14
1,1,1,0.604554,0.604554,-54.165000,nul,90.333,C11H23N5O3,[M + H]+,C11H24N5O3+,...,CC(C)C(C(=O)O)NC(=O)C(CCCN=C(N)N)N,-1.217,4278572;6992653;6992654;154470066;140711172;92...,COCONUT:(CNP0347645);HMDB:(HMDB0028722);BloodE...,138251608174,274.187,22,567749695871841810,567749454275726814,28
2,1,1,0.225944,0.225944,-262.099951,nul,29.940,C12H21N5O5,[M + Na]+,C12H21N5NaO5+,...,CC(=O)NC(CCC(=O)N)C(=O)NC(CCC(=O)N)C(=O)N,-3.033,22673233;129914,PUBCHEMANNOTATIONBIO;METACYC:(CPD-14436);CHEBI...,137489287270,338.143,19,567749669149930142,567749454372195810,33
3,1,1,0.946329,0.946329,-54.246682,nul,45.229,C10H20N2O4S,[M + H]+,C10H21N2O4S+,...,CC(C)C(C(=O)NC(CCS(=O)C)C(=O)O)N,-1.310,91746311,SUPERNATURAL:(SN0157389);COCONUT:(CNP0347436);...,3145730,265.122,20,567749657737228665,567749454531579370,45
4,1,1,0.416415,0.416415,-118.903767,nul,46.451,C16H30N6O4,[M + H]+,C16H31N6O4+,...,CC(C)C(C(=O)N1CCCC1C(=O)NC(CCCN=C(N)N)C(=O)O)N,-1.043,10248593;18232480,PUBCHEM:(10248593 18232480);CHEBI:(166336),34,371.240,20,567749662992691702,567749454649019886,55


In [17]:
prepare_fbmn_annotation_for_cytoscape(sirius_annotations_path, 'featureId', tool)

Removed 2 duplicates based on featureId, name, and smiles
FeatureID is unique. No aggregation needed.
No columns found containing the substring 'Compound_name'.
Required Sirius columns are not all present.
File saved as: results_download/sirius/summary-files/structure_identifications_prep.tsv


,featureId,sir_struct_structurePerIdRank,sir_struct_formulaRank,sir_struct_ConfidenceScoreExact,sir_struct_ConfidenceScoreApproximate,sir_struct_CSI:FingerIDScore,sir_struct_ZodiacScore,sir_struct_SiriusScore,sir_struct_molecularFormula,sir_struct_adduct,...,sir_struct_xlogp,sir_struct_pubchemids,sir_struct_links,sir_struct_dbflags,sir_struct_ionMass,sir_struct_retentionTimeInSeconds,sir_struct_formulaId,sir_struct_alignedFeatureId,sir_struct_smiles,annotation_tool
0,14,1,1,0.739409,0.739409,-15.718746,nul,38.171,C9H18N2O4,[M + H]+,...,-0.997,4046492;9815826;124306824;124422925;156908028;...,COCONUT:(CNP0292075);PUBCHEMANNOTATIONBIO;HMDB...,556793898,219.134,20,567749660631298519,567749454112148954,CC(C)C(C(=O)NC(C(C)O)C(=O)O)N,sir_struct
1,28,1,1,0.604554,0.604554,-54.165000,nul,90.333,C11H23N5O3,[M + H]+,...,-1.217,4278572;6992653;6992654;154470066;140711172;92...,COCONUT:(CNP0347645);HMDB:(HMDB0028722);BloodE...,138251608174,274.187,22,567749695871841810,567749454275726814,CC(C)C(C(=O)O)NC(=O)C(CCCN=C(N)N)N,sir_struct
2,33,1,1,0.225944,0.225944,-262.099951,nul,29.940,C12H21N5O5,[M + Na]+,...,-3.033,22673233;129914,PUBCHEMANNOTATIONBIO;METACYC:(CPD-14436);CHEBI...,137489287270,338.143,19,567749669149930142,567749454372195810,CC(=O)NC(CCC(=O)N)C(=O)NC(CCC(=O)N)C(=O)N,sir_struct
3,45,1,1,0.946329,0.946329,-54.246682,nul,45.229,C10H20N2O4S,[M + H]+,...,-1.310,91746311,SUPERNATURAL:(SN0157389);COCONUT:(CNP0347436);...,3145730,265.122,20,567749657737228665,567749454531579370,CC(C)C(C(=O)NC(CCS(=O)C)C(=O)O)N,sir_struct
4,55,1,1,0.416415,0.416415,-118.903767,nul,46.451,C16H30N6O4,[M + H]+,...,-1.043,10248593;18232480,PUBCHEM:(10248593 18232480);CHEBI:(166336),34,371.240,20,567749662992691702,567749454649019886,CC(C)C(C(=O)N1CCCC1C(=O)NC(CCCN=C(N)N)C(=O)O)N,sir_struct


### TIMA annotations

There is a `feature_id` column. 

Lets do some bonus formatting

In [18]:
tool = 'tima'
tima_annotations_path = 'results_download/tima/240414_114832_comp_ms_prague/comp_ms_prague_results.tsv'
tima_annotations = pd.read_csv(tima_annotations_path, sep='\t')
tima_annotations.head(5)

,feature_id,feature_mz,feature_rt,feature_spectrum_entropy,feature_pred_tax_cla_01kin_val,feature_pred_tax_cla_01kin_score,feature_pred_tax_cla_02sup_val,feature_pred_tax_cla_02sup_score,feature_pred_tax_cla_03cla_val,feature_pred_tax_cla_03cla_score,...,candidate_structure_organism_occurrence_closest,candidate_structure_organism_occurrence_reference,candidate_structure_error_mz,candidate_structure_error_rt,rank_initial,rank_final,score_initial,score_biological,score_chemical,score_final
0,14,219.133758,0.387136,1.352125,NaN,NaN,Organic acids and derivatives,1.000000,Carboxylic acids and derivatives,0.999999,...,NaN,NaN,0.000256491909453871,NaN,1,1,0.636183070172288,0,0,0.212061023390763
1,28,274.187123,0.368677,1.116997,NaN,NaN,Organic acids and derivatives,0.999984,Carboxylic acids and derivatives,0.999927,...,NaN,NaN,0.000265596000019741,NaN,1,1,0.345558285713196,0,1,0.281352261403898
2,33,338.143377,0.327985,2.590523,NaN,NaN,Organic acids and derivatives,0.999999,Carboxylic acids and derivatives,0.999990,...,Bacteria,NaN,0.00414811799993231,NaN,17,1,0,0.1,1,0.216216216216216
3,36,653.297125,0.318666,2.495411,Organic compounds,0.00000,Lipids and lipid-like molecules,0.000000,Prenol lipids,0.000000,...,NaN,NaN,-0.00761400399983359,NaN,1,1,0,0,1,0.166166166166166
4,37,354.116550,0.347539,2.144528,Organic compounds,0.13814,Organic acids and derivatives,0.103605,Carboxylic acids and derivatives,0.103605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
prepare_fbmn_annotation_for_cytoscape(tima_annotations_path, 'feature_id', tool)

Neither 'compound_name' nor 'name' column is present. Checking for duplicates based on SMILES only.
Removed 0 duplicates based on feature_id and candidate_structure_smiles_no_stereo
FeatureID is unique. No aggregation needed.
No columns found containing the substring 'Compound_name'.
File saved as: results_download/tima/240414_114832_comp_ms_prague/comp_ms_prague_results_prep.tsv


,feature_id,tima_feature_mz,tima_feature_rt,tima_feature_spectrum_entropy,tima_feature_pred_tax_cla_01kin_val,tima_feature_pred_tax_cla_01kin_score,tima_feature_pred_tax_cla_02sup_val,tima_feature_pred_tax_cla_02sup_score,tima_feature_pred_tax_cla_03cla_val,tima_feature_pred_tax_cla_03cla_score,...,tima_rank_initial,tima_rank_final,tima_score_initial,tima_score_biological,tima_score_chemical,tima_score_final,tima_candidate_structure_smiles_no_stereo,annotation_tool,name,smiles
0,14,219.133758,0.387136,1.352125,NaN,NaN,Organic acids and derivatives,1.000000,Carboxylic acids and derivatives,0.999999,...,1,1,0.636183070172288,0,0,0.212061023390763,CC(C)C(C(=O)NC(C(C)O)C(=O)O)N,tima,NaN,CC(C)C(C(=O)NC(C(C)O)C(=O)O)N
1,28,274.187123,0.368677,1.116997,NaN,NaN,Organic acids and derivatives,0.999984,Carboxylic acids and derivatives,0.999927,...,1,1,0.345558285713196,0,1,0.281352261403898,CC(C)C(N)C(=O)NC(CCCNC(=N)N)C(=O)O,tima,Valyl-arginine | Dipeptides,CC(C)C(N)C(=O)NC(CCCNC(=N)N)C(=O)O
2,33,338.143377,0.327985,2.590523,NaN,NaN,Organic acids and derivatives,0.999999,Carboxylic acids and derivatives,0.999990,...,17,1,0,0.1,1,0.216216216216216,CCC(NC(=O)C(C)NC(=O)C(N)CCP(C)(=O)O)C(=O)O,tima,NaN,CCC(NC(=O)C(C)NC(=O)C(N)CCP(C)(=O)O)C(=O)O
3,36,653.297125,0.318666,2.495411,Organic compounds,0.00000,Lipids and lipid-like molecules,0.000000,Prenol lipids,0.000000,...,1,1,0,0,1,0.166166166166166,CC(C)(O)C1CCC(C)(C(O)CCC(O)(CO)C(=O)C(O)CC2OC(...,tima,"1-[6-(5-Bromo-2,6,6-trimethyloxan-2-yl)-3-hydr...",CC(C)(O)C1CCC(C)(C(O)CCC(O)(CO)C(=O)C(O)CC2OC(...
4,37,354.116550,0.347539,2.144528,Organic compounds,0.13814,Organic acids and derivatives,0.103605,Carboxylic acids and derivatives,0.103605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tima,NaN,NaN


## We continue with Cytoscape

Exploration and style

## We download all the files for Cytoscape

In [30]:
import os
import zipfile
from IPython.display import display, HTML

def zip_prep_files(directory, zip_name, depth=5):
    """
    Search for all files ending in _prep.tsv within the given depth of the directory
    and make a zip archive out of them.

    :param directory: The directory to search for files in.
    :param zip_name: The name of the output zip file.
    :param depth: The depth to search for files. If -1, search all levels.
                  Depth of 0 means the current directory only,
                  1 means the current directory and its immediate subdirectories, and so on.
    """
    def should_include_dir(root_depth, current_depth):
        # If depth is negative, no limit is applied.
        if depth < 0:
            return True
        # Include directories within the desired depth.
        return (current_depth - root_depth) <= depth

    root_depth = directory.count(os.sep)
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            current_depth = root.count(os.sep)
            if should_include_dir(root_depth, current_depth):
                for file in files:
                    if file.endswith('_prep.tsv'):
                        print(file)
                        filepath = os.path.join(root, file)
                        zipf.write(filepath, os.path.relpath(filepath, start=directory))
                # Modify the dirs in place to avoid unnecessary recursion into subdirectories beyond the depth
                if not should_include_dir(root_depth, current_depth + 1):
                    dirs.clear()  # This prevents os.walk from going into deeper directories
    print(f"Created zip archive: {zip_name}")
    file_link = f'<a href="{zip_name}" download="{zip_name}">Click here to download {zip_name} the zip archive</a>'
    display(HTML(file_link))


# Example usage:
# Provide the directory to search in, the desired zip file name, and the depth
# zip_prep_files('/path/to/directory', 'prep_files_archive.zip', depth=1)  # Adjust the depth as needed



In [31]:
zip_prep_files('results_download', 'cytoscape_input.zip')

canopus_structure_summary_prep.tsv
structure_identifications_prep.tsv
comp_ms_prague_results_prep.tsv
ea4293bedd5440148267cb201ef7edbc-merged_query_results_MassQL_prep.tsv
861f707d5a4f42e88486c77a4693a38d-merged_results_with_gnps_prep.tsv
ms2query_results_for_cytoscape_prep.tsv
mzmine_results_iimn_gnps_edges_msannotation_prep.tsv
mzmine_results_annotations_prep.tsv
Created zip archive: cytoscape_input.zip
